In [88]:
import pandas
import konlpy
import gensim

print(pandas.__version__)
print(konlpy.__version__)
print(gensim.__version__)

1.3.3
0.5.2
4.1.2


# 1) 연습

In [11]:
import pandas as pd

# 데이터를 읽어봅시다. 
train_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_train.txt')
test_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_test.txt')

print(len(train_data))
print(len(test_data))
train_data.head()


150000
50000


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [12]:
# 중복 데이터는 없는것같다.
print(len(train_data.drop_duplicates()))
print(len(test_data.drop_duplicates()))

150000
50000


In [18]:
# NaN이 포함된 행만 출력
print(train_data[train_data.isna().any(axis=1)])
print(test_data[test_data.isna().any(axis=1)])

             id document  label
25857   2172111      NaN      1
55737   6369843      NaN      1
110014  1034280      NaN      0
126782  5942978      NaN      0
140721  1034283      NaN      0
            id document  label
5746    402110      NaN      1
7899   5026896      NaN      0
27097   511097      NaN      1


In [20]:
# 결측치 제거
train_data = train_data.dropna(subset=['document'])
test_data = test_data.dropna(subset=['document'])

print(len(train_data))
print(len(test_data))

149995
49997


In [27]:
# 한국어 토크나이저로 토큰화
from konlpy.tag import Mecab

tokenizer = Mecab()

# 각 행에 대해 형태소 분석 수행
train_data['tokens'] = train_data['document'].apply(tokenizer.morphs)
test_data['tokens'] = test_data['document'].apply(tokenizer.morphs)

In [28]:
train_data.head()

,id,document,label,tokens
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,"[아, 더, 빙, ., ., 진짜, 짜증, 나, 네요, 목소리]"
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,"[흠, ., .., 포스터, 보고, 초딩, 영화, 줄, ., ..., 오버, 연기,..."
2,10265843,너무재밓었다그래서보는것을추천한다,0,"[너무, 재, 밓었다그래서보는것을추천한다]"
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,"[교도소, 이야기, 구먼, ., ., 솔직히, 재미, 는, 없, 다, ., ., 평..."
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,"[사이몬페그, 의, 익살, 스런, 연기, 가, 돋보였, 던, 영화, !, 스파이더맨..."


In [29]:
test_data.head()

,id,document,label,tokens
0,6270596,굳 ㅋ,1,"[굳, ㅋ]"
1,9274899,GDNTOPCLASSINTHECLUB,0,[GDNTOPCLASSINTHECLUB]
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,"[뭐, 야, 이, 평점, 들, 은, ., ..., 나쁘, 진, 않, 지만, 10, ..."
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,"[지루, 하, 지, 는, 않, 은데, 완전, 막장, 임, ., .., 돈, 주, 고..."
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,"[3, D, 만, 아니, 었, 어도, 별, 다섯, 개, 줬, 을, 텐데, ., .,..."


In [30]:
# 불용어(Stopwords) 제거
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# 불용어 제거 함수 정의
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stopwords]

train_data['tokens'] = train_data['tokens'].apply(remove_stopwords)
test_data['tokens'] = test_data['tokens'].apply(remove_stopwords)

In [32]:
train_data.head()

,id,document,label,tokens
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,"[아, 더, 빙, ., ., 진짜, 짜증, 나, 네요, 목소리]"
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,"[흠, ., .., 포스터, 보고, 초딩, 영화, 줄, ., ..., 오버, 연기,..."
2,10265843,너무재밓었다그래서보는것을추천한다,0,"[너무, 재, 밓었다그래서보는것을추천한다]"
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,"[교도소, 이야기, 구먼, ., ., 솔직히, 재미, 없, 다, ., ., 평점, 조정]"
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,"[사이몬페그, 익살, 스런, 연기, 돋보였, 던, 영화, !, 스파이더맨, 에서, ..."


In [33]:
test_data.head()

,id,document,label,tokens
0,6270596,굳 ㅋ,1,"[굳, ㅋ]"
1,9274899,GDNTOPCLASSINTHECLUB,0,[GDNTOPCLASSINTHECLUB]
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,"[뭐, 야, 평점, ., ..., 나쁘, 진, 않, 지만, 10, 점, 짜리, 더더..."
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,"[지루, 하, 지, 않, 은데, 완전, 막장, 임, ., .., 돈, 주, 고, 보..."
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,"[3, D, 만, 아니, 었, 어도, 별, 다섯, 개, 줬, 을, 텐데, ., .,..."


In [52]:
# 사전 word_to_index 구성
index_to_word = {}
index_to_word[0] = '<PAD>'  # 패딩용 단어
index_to_word[1] = '<BOS>'  # 문장의 시작지점
index_to_word[2] = '<UNK>'  # 사전에 없는(Unknown) 단어

# 리스트 컴프리헨션으로 모든 리스트의 문자열 데이터를 하나의 리스트에 담음
all_tokens = [token for tokens in train_data['tokens'] for token in tokens]

# 중복 제거 및 리스트 변환 (한 번만 실행)
all_tokens_list = list(set(all_tokens))

# 인덱싱
for i, token in enumerate(all_tokens_list, start=3):
    index_to_word[i] = token

print(len(index_to_word))

# iword_to_index
word_to_index = {word:index for index, word in index_to_word.items()}
print(len(word_to_index))

53964
53964


In [58]:
# 문장 인코딩 함수
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']] + [
        word_to_index[word] if word in word_to_index else word_to_index['<UNK>']
        for word in sentence
    ]

# 적용
train_data['encoded_sentence'] = train_data['tokens'].apply(
    lambda tokens: get_encoded_sentence(tokens, word_to_index)
)
# 적용
test_data['encoded_sentence'] = test_data['tokens'].apply(
    lambda tokens: get_encoded_sentence(tokens, word_to_index)
)


In [59]:
train_data.head()

,id,document,label,tokens,encoded_sentence
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,"[아, 더, 빙, ., ., 진짜, 짜증, 나, 네요, 목소리]","[1, 23228, 33893, 24420, 37899, 37899, 39791, ..."
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,"[흠, ., .., 포스터, 보고, 초딩, 영화, 줄, ., ..., 오버, 연기,...","[1, 25521, 37899, 23163, 41085, 49336, 5304, 2..."
2,10265843,너무재밓었다그래서보는것을추천한다,0,"[너무, 재, 밓었다그래서보는것을추천한다]","[1, 51749, 15612, 20103]"
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,"[교도소, 이야기, 구먼, ., ., 솔직히, 재미, 없, 다, ., ., 평점, 조정]","[1, 3925, 16976, 7700, 37899, 37899, 11472, 46..."
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,"[사이몬페그, 익살, 스런, 연기, 돋보였, 던, 영화, !, 스파이더맨, 에서, ...","[1, 8972, 29478, 48460, 18532, 41108, 31820, 2..."


In [60]:
test_data.head()

,id,document,label,tokens,encoded_sentence
0,6270596,굳 ㅋ,1,"[굳, ㅋ]","[1, 21340, 4777]"
1,9274899,GDNTOPCLASSINTHECLUB,0,[GDNTOPCLASSINTHECLUB],"[1, 2]"
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,"[뭐, 야, 평점, ., ..., 나쁘, 진, 않, 지만, 10, 점, 짜리, 더더...","[1, 38417, 25070, 11938, 37899, 51094, 26550, ..."
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,"[지루, 하, 지, 않, 은데, 완전, 막장, 임, ., .., 돈, 주, 고, 보...","[1, 33393, 50599, 10734, 36695, 38557, 26316, ..."
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,"[3, D, 만, 아니, 었, 어도, 별, 다섯, 개, 줬, 을, 텐데, ., .,...","[1, 51341, 24568, 1170, 28113, 32467, 39930, 3..."


# 2) 데이터로더 구성


* 데이터의 중복 제거
* NaN 결측치 제거
* 한국어 토크나이저로 토큰화
* 불용어(Stopwords) 제거
* 사전word_to_index 구성
* 텍스트 스트링을 사전 인덱스 스트링으로 변환
* X_train, y_train, X_test, y_test, word_to_index 리턴

In [104]:
from konlpy.tag import Mecab
import numpy as np
from collections import Counter

import pandas as pd

# 데이터를 읽어봅시다. 
train_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_train.txt')
test_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_test.txt')

tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']


# 불용어 제거 함수 정의
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stopwords]


# 문장 인코딩 함수
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']] + [
        word_to_index[word] if word in word_to_index else word_to_index['<UNK>']
        for word in sentence
    ]


def load_data(train_data, test_data):
# ----------------------------------------------------------------------
#      중복 데이터 제거
# ----------------------------------------------------------------------
    train_data = train_data.drop_duplicates()
    test_data = test_data.drop_duplicates()
    
# ----------------------------------------------------------------------
#      NaN결측치 제거
# ----------------------------------------------------------------------
    train_data = train_data.dropna(subset=['document'])
    test_data = test_data.dropna(subset=['document'])
    
# ----------------------------------------------------------------------
#      한국어 토크나이저로 토큰화
# ----------------------------------------------------------------------
    tokenizer = Mecab()
    train_data['tokens'] = train_data['document'].apply(tokenizer.morphs)
    test_data['tokens'] = test_data['document'].apply(tokenizer.morphs)
    
# ----------------------------------------------------------------------
#      불용어 제거
# ----------------------------------------------------------------------
    train_data['tokens'] = train_data['tokens'].apply(remove_stopwords)
    test_data['tokens'] = test_data['tokens'].apply(remove_stopwords)
    
# ----------------------------------------------------------------------
#      사전 word_to_index 구성
# ----------------------------------------------------------------------
    index_to_word = {}
    index_to_word[0] = '<PAD>'  # 패딩용 단어
    index_to_word[1] = '<BOS>'  # 문장의 시작지점
    index_to_word[2] = '<UNK>'  # 사전에 없는(Unknown) 단어

    # 리스트 컴프리헨션으로 모든 리스트의 문자열 데이터를 하나의 리스트에 담음
    all_tokens = [token for tokens in train_data['tokens'] for token in tokens]

    # 중복 제거 및 리스트 변환 (한 번만 실행)
    all_tokens_list = list(set(all_tokens))

    # 인덱싱
    for i, token in enumerate(all_tokens_list, start=3):
        index_to_word[i] = token

    # word_to_index
    word_to_index = {word:index for index, word in index_to_word.items()}
    
# ----------------------------------------------------------------------
#      텍스트 스트링을 사전 인덱스 스트링으로 변환
# ----------------------------------------------------------------------    
    
    # 적용
    train_data['encoded_sentence'] = train_data['tokens'].apply(
        lambda tokens: get_encoded_sentence(tokens, word_to_index)
    )
    # 적용
    test_data['encoded_sentence'] = test_data['tokens'].apply(
        lambda tokens: get_encoded_sentence(tokens, word_to_index)
    )
    
# ----------------------------------------------------------------------
#      X_train, y_train, X_test, y_test, word_to_index 리턴
# ----------------------------------------------------------------------
    X_train = train_data['encoded_sentence']
    y_train = train_data['label']
    X_test = test_data['encoded_sentence']
    y_test = test_data['label']
    
    return X_train, y_train, X_test, y_test, word_to_index
    
X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data)

# 3) 모델 구성을 위한 데이터 분석 및 가공
* 데이터셋 내 문장 길이 분포
* 적절한 최대 문장 길이 지정
* keras.preprocessing.sequence.pad_sequences 을 활용한 패딩 추가

In [105]:
type(X_train)
X_train.head()

0    [1, 23228, 33893, 24420, 37899, 37899, 39791, ...
1    [1, 25521, 37899, 23163, 41085, 49336, 5304, 2...
2                             [1, 51749, 15612, 20103]
3    [1, 3925, 16976, 7700, 37899, 37899, 11472, 46...
4    [1, 8972, 29478, 48460, 18532, 41108, 31820, 2...
Name: encoded_sentence, dtype: object

In [110]:
import tensorflow as tf

vocab_size = len(word_to_index)  # 위 예시에서 딕셔너리에 포함된 단어 개수는 10

word_vector_dim = 20    # 그림과 같이 4차원의 워드 벡터를 가정합니다.

embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=word_vector_dim, mask_zero=True)

# 최대길이 : 117
# 패딩 추가
raw_inputs_train = np.array(X_train, dtype=object)
raw_inputs_train = tf.keras.preprocessing.sequence.pad_sequences(raw_inputs_train,
                                                       value=word_to_index['<PAD>'],
                                                       padding='post',
                                                       maxlen=117)
raw_inputs_test = np.array(X_test, dtype=object)
raw_inputs_test = tf.keras.preprocessing.sequence.pad_sequences(raw_inputs_test,
                                                       value=word_to_index['<PAD>'],
                                                       padding='post',
                                                       maxlen=117)

output_train = embedding(raw_inputs_train)
output_test = embedding(raw_inputs_test)

In [85]:
print(len(output_train[0]))
print(len(output_test[0]))

print(output_train[0])
print(output_test[0])

117
117
tf.Tensor(
[[ 0.03245324  0.04827912  0.0345708  ...  0.01533706  0.04945772
  -0.00353019]
 [ 0.04544746  0.01817102  0.02516193 ... -0.01255883 -0.02149509
  -0.02839286]
 [ 0.0209243   0.03748203 -0.03297709 ...  0.03218491  0.02320417
  -0.01926425]
 ...
 [-0.04095118  0.0027527   0.0153636  ...  0.02924586  0.00105881
  -0.04945065]
 [-0.04095118  0.0027527   0.0153636  ...  0.02924586  0.00105881
  -0.04945065]
 [-0.04095118  0.0027527   0.0153636  ...  0.02924586  0.00105881
  -0.04945065]], shape=(117, 20), dtype=float32)
tf.Tensor(
[[ 0.03245324  0.04827912  0.0345708  ...  0.01533706  0.04945772
  -0.00353019]
 [ 0.01615688 -0.00917904  0.03366959 ...  0.02129573  0.00050046
  -0.00033401]
 [-0.02488056 -0.01137577 -0.00290271 ... -0.00480497  0.01973649
   0.03806305]
 ...
 [-0.04095118  0.0027527   0.0153636  ...  0.02924586  0.00105881
  -0.04945065]
 [-0.04095118  0.0027527   0.0153636  ...  0.02924586  0.00105881
  -0.04945065]
 [-0.04095118  0.0027527   0.015363

# 4) 모델 구성 및 validation set 구성
모델은 3가지 이상 다양하게 구성하여 실험해 보세요.

In [90]:
model1 = tf.keras.Sequential()
model1.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None, )))
model1.add(tf.keras.layers.LSTM(8))  # LSTM 레이어
model1.add(tf.keras.layers.Dense(8, activation='relu'))  # Dense 레이어
model1.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력 레이어

model1.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 20)          1079280   
_________________________________________________________________
lstm (LSTM)                  (None, 8)                 928       
_________________________________________________________________
dense (Dense)                (None, 8)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 1,080,289
Trainable params: 1,080,289
Non-trainable params: 0
_________________________________________________________________


In [91]:
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model2.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model2.add(tf.keras.layers.MaxPooling1D(5))
model2.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model2.add(tf.keras.layers.GlobalMaxPooling1D())
model2.add(tf.keras.layers.Dense(8, activation='relu'))
model2.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 20)          1079280   
_________________________________________________________________
conv1d (Conv1D)              (None, None, 16)          2256      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 16)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 16)          1808      
_________________________________________________________________
global_max_pooling1d (Global (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [92]:
model3 = tf.keras.Sequential()
model3.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model3.add(tf.keras.layers.GlobalMaxPooling1D())
model3.add(tf.keras.layers.Dense(8, activation='relu'))
model3.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model3.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 20)          1079280   
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 20)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 168       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 9         
Total params: 1,079,457
Trainable params: 1,079,457
Non-trainable params: 0
_________________________________________________________________


In [118]:
# validation set 
X_val = output_train[:30000]
partial_X_train = output_train[30000:]

y_val = np.array(y_train[:30000], dtype=object)
partial_y_train = np.array(y_train[30000:], dtype=object)

print(partial_X_train.shape)
print(partial_y_train.shape)

print(type(X_val))
print(type(partial_X_train))
print(type(y_val))
print(type(partial_y_train))


(119995, 117, 20)
(119995,)
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [114]:
model1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

epochs=20  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다.

history = model1.fit(partial_X_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(X_val, y_val),
                    verbose=1)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).